`pd_sc` - Helpful Engineering's Project Data Supply Chain modeling and CLI

> **License info:**
> 
> Copyright (C) 2021  Robert L. Read <read.robert@gmail.com>
> 
> This program is free software: you can redistribute it and/or modify it under the terms of the GNU Affero General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.
> 
> This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU Affero General Public License for more details.
> 
> You should have received a copy of the GNU Affero General Public License along with this program.  If not, see <https://www.gnu.org/licenses/>.


*This notebook is incomplete! These markdown blocks are useful for longform explanations.*

In [23]:
import copy
import functools as ft
import pprint
import unittest

import sympy as sp

# Instantiate JupyterLab from project root
# or you will get "ModuleNotFoundError: No module named 'src'
import src.supply as supply
import src.okf as okf
import src.stage_graph as stage_graph

Our most basic type is the "Product Type", but we will just use strings for these for now.

The second type is the Supply, which is really short for "SupplyCapability". A Supply has inputs and output types, a name, and a characteristic equation.

Here, we create a very basic set of supplies. First, we will create "symbols" (similar to lisp 'intern').

In [5]:
chair, leg, seat, back = supply.symbols("chair leg seat back")
fabric, plane = supply.symbols("fabric plane")
frame, stuffing = supply.symbols("frame stuffing")
upholstery = supply.symbols("upholstery")

# These are more or "supplier" constants
chair_1, chair_2, leg_1, seat_1, seat_2, seat_3 = supply.symbols("chair_1 chair_2 leg_1 seat_1 seat_2 seat_3")
back_1, fabric_1, plane_1 = supply.symbols("back_1 fabric_1 plane_1")
fabric_2 = supply.symbols("fabric_2")
stuffing_1 = supply.symbols("stuffing_1")

In [6]:
# This makes more sense as a dictionary;
# However, the SymPy package uses a list of pairs (symbol, expression).
price_map = [
    (chair_1, 4),
    (chair_2, 3),
    (leg_1, 1),
    (seat_1, 2),
    (seat_2, 2),
    (seat_3, 3),
    (back_1, 3),
    (fabric_1, 1),
    (plane_1, 1),
    (stuffing_1, 2)
]

todo! others might add better markdown blocks to explain these 

In [8]:
# Possibly we should create anonymous supply signatures...
# These are effectively hard-coded OKH elements; we may
# eventually have a way to import OKHs.
# TODO: Shift seat to "Frame, stuffing, upholstery"
# TODO: Shift to a mask
c1  = supply.Supply("chair_1",["chair"],["leg","seat","back"],chair_1 + 4*leg + seat + back)
c2  = supply.Supply("chair_2",["chair"],["leg","seat","back"],chair_2 + 4*leg + seat + back)
l1  = supply.Supply("leg_1",["leg"],[],leg_1)
s1  = supply.Supply("seat_1",["seat"],[],seat_1)
s2  = supply.Supply("seat_2",["seat"],["fabric","plane"],seat_2 + fabric + plane)
s3  = supply.Supply("seat_3",["seat"],["frame","stuffing","upholstery"],seat_3 + frame + stuffing + upholstery)
ss1 = supply.Supply("stuffing_1",["stuffing"],[],stuffing_1)
b1  = supply.Supply("back_1",["back"],[],back_1)
f1  = supply.Supply("fabric_1",["fabric"],[],fabric_1)
f2  = supply.Supply("fabric_2",["fabric"],[],fabric_2)
p1  = supply.Supply("plane_1",["plane"],[],plane_1)

a = supply.SupplyNetwork("A",[c1,c2,l1,s1,b1,s2,f1,p1,s3,ss1])

A,B,C,X,Y = supply.symbols("A B C X Y")
A_1,B_1,C_1,X_1,Y_1 = supply.symbols("A_1 B_1 C_1 X_1 Y_1")

A1 = supply.Supply("A_1",["A"],["B","C"],A_1 + B + C )
B1 = supply.Supply("B_1",["B"],[],B_1)
C1 = supply.Supply("C_1",["C"],[],C_1)

X1 = supply.Supply("X_1",["X"],["Y"],X_1+Y)
Y1 = supply.Supply("Y_1",["Y"],[],Y_1)

xy = supply.SupplyNetwork("XY",[X1,Y1])
ab = supply.SupplyNetwork("AB",[A1,B1])
abc = supply.SupplyNetwork("ABC",[A1,B1,C1])

In [9]:
# SupplyTree is consistent only if the Supplies in the inputDict match the type
sx = supply.SupplyTree(
    c1,
    {
        "leg":  supply.SupplyTree(l1,{}),
        "seat": supply.SupplyTree(s1,{}),
        "back": supply.SupplyTree(b1,{})
    }
)

In [12]:
# Print only those SupplyTrees which are complete in the "chair" problem
for st in list(supply.SupplyProblem("chair",a).completeSupplyTrees()):
    print(st)
    print(supply.characteristicExpression(st))
    print(supply.characteristicExpression(st).subs(price_map))

          chair:chair_1
---------------------------------
back:back_1,seat:seat_1,leg:leg_1
back:back_1
seat:seat_1
leg:leg_1

back_1 + chair_1 + 4*leg_1 + seat_1
13
          chair:chair_1
---------------------------------
back:back_1,seat:seat_2,leg:leg_1
back:back_1
         seat:seat_2
-----------------------------
plane:plane_1,fabric:fabric_1
plane:plane_1
fabric:fabric_1
leg:leg_1

back_1 + chair_1 + fabric_1 + 4*leg_1 + plane_1 + seat_2
15
          chair:chair_2
---------------------------------
back:back_1,seat:seat_1,leg:leg_1
back:back_1
seat:seat_1
leg:leg_1

back_1 + chair_2 + 4*leg_1 + seat_1
12
          chair:chair_2
---------------------------------
back:back_1,seat:seat_2,leg:leg_1
back:back_1
         seat:seat_2
-----------------------------
plane:plane_1,fabric:fabric_1
plane:plane_1
fabric:fabric_1
leg:leg_1

back_1 + chair_2 + fabric_1 + 4*leg_1 + plane_1 + seat_2
14


In [15]:
print(
    supply.SupplyProblem("chair",a).optimalCompleteSupplyTreeByPrice(price_map)[0]
)


          chair:chair_2
---------------------------------
back:back_1,seat:seat_2,leg:leg_1
back:back_1
         seat:seat_2
-----------------------------
plane:plane_1,fabric:fabric_1
plane:plane_1
fabric:fabric_1
leg:leg_1



---

Todo! Put some text here explaining what the code is doing how with the OKH, OKW, OKF.

---

In [16]:
# python has 'keyword arguments', useful here for clarity
okh1 = okf.OKH(
    name = "SurgeMask",
    outputs = ["mask"],
    inputs = ["NWPP", "coffee_tin_ties", "fabric_ties"],
    requiredTooling = ["sewing_machine"],
    eqn = "no eqn yet"
)

In [18]:
okw1 = okf.OKW(
    name = "NearJames",
    toolingGoods = ["sewing_machine"]
)

In [19]:
okf1 = okf.OKF(
    name = "okf1",
    okhs = [okh1],
    okws = [okw1]
)

---

Now create a SupplyNetwork from the okf, and union it with our non-OKF supplies..

In [20]:
okf_sn = supply.SupplyNetwork("fromOKF",okf1.supplies())
combined = supply.unionSupplyNetworks(a,okf_sn)

for st in list(supply.SupplyProblem("mask",okf_sn)):
    print(st)

mask:NearJames|SurgeMask



In [21]:
sgc = stage_graph.StageGraph("chair",sx)
sgc.assertSupplyStatus("seat_1",StageStatus.FAILED)
print(sgc.isComplete())
print(sgc.needsRepair())
print(sgc.nameOfSupplyThatNeedsRepair())

NameError: name 'stage_graph' is not defined